In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from bs4 import BeautifulSoup
import torchvision
from torchvision import transforms, datasets, models
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import matplotlib.patches as patches
import os
from torch.utils.data import Dataset

In [2]:
data_transform = transforms.Compose([transforms.ToTensor(), ])
#root = '/content/drive/MyDrive/Diplomado Aplicaciones IA/Visión Computacional'
root = '/content/drive/MyDrive/Visión Computacional/Proyecto Final/'
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [4]:
def get_model_instance_segmentation(num_classes):
  # load an instance segmentation model pre-trained pre-trained on COCO
  model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
  # get number of input features for the classifier
  in_features = model.roi_heads.box_predictor.cls_score.in_features
  # replace the pre-trained head with a new one
  model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes+1)

  return model

In [5]:
model = get_model_instance_segmentation(3)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


In [13]:
model.load_state_dict(torch.load(root + 'Models/model_zombie_5.pth'))
model.eval()
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256)
          (relu): ReLU(inplace=True)
          (downsample)

In [7]:
import cv2
from google.colab.patches import cv2_imshow

In [8]:
root

'/content/drive/MyDrive/Visión Computacional/Proyecto Final/'

In [14]:
video_input ='tam'
video_output = 'tam_model_5'
video = cv2.VideoCapture(root + 'Videos/'+video_input+'.mp4')
writer=None

while True:

  (grab, frame)=video.read()
  #cv2_imshow(frame)
  if writer is None:
    fourcc = cv2.VideoWriter_fourcc('F', 'M', 'P', '4')
    print(int(video.get(3)),int(video.get(4)))
    writer = cv2.VideoWriter(root + 'Videos/'+video_output+'.avi',fourcc, 30,(int(video.get(3)),int(video.get(4))), True)
  if not grab:
    break

  image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  #image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE) 
  image_tensor = data_transform(image)
  image_tensor=image_tensor.to(device)
  image_tensor = image_tensor.unsqueeze(0)
  predicted_bb = model(image_tensor)
  predicted_bb =predicted_bb[0]
  #print(predicted_bb)
  if predicted_bb is not bool(predicted_bb):
    #print(predicted_bb["boxes"])
    box_label_score = zip( predicted_bb["boxes"],predicted_bb["labels"] , predicted_bb['scores'])
    for box, label ,score in box_label_score:
      if score >0.75:
        if label ==2:
          color = (0,255,0)
        elif label ==3:
          color = (255,255,0)
        elif label ==1:
          color = (255,0,0)
        #print(box)
        xmin, ymin, xmax, ymax = box
        start = (xmin,ymin)
        end = (xmax,ymax)
        image=cv2.rectangle(image, start,end,color,2)
      #cv2_imshow(cv2.cvtColor(image, cv2.COLOR_RGB2BGR))   
    writer.write(cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
  else:
    writer.write(cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
writer.release()

480 848
